In [11]:
import pandas as pd
import re

## Data

In [12]:
KATEGLO = pd.read_csv('data/kateglo.csv')[['lema', 'nilai', 'kelas']].fillna('')
DICT = {k: v for _,k,v in KATEGLO[['lema', 'kelas']].to_records()}

In [3]:
# TODO: https://stackoverflow.com/questions/42742810/speed-up-millions
# -of-regex-replacements-in-python-3/42789508#42789508

#KATA[KATA.lema.apply(lambda x: '.' in str(x))] ==> empty
with open('data/singkatan.txt', 'r') as f:
    ABBREV = set(f.read().split('\n')) - {''}

In [4]:
# https://news.detik.com/berita-jawa-tengah/d-5589430/kondisi-terkini-eks-pimpinan-yang-tersengal-sengal-di-gedung-dprd-klaten
TEXT = '''Klaten - Pimpinan DPRD Klaten periode 2005-2010, Anang Widayaka dievakuasi dari gedung DPRD Klaten dalam kondisi napas tersengal-sengal kemarin. Saat ini kondisi Anang disebut sudah membaik.
"Tadi malam saya mendapat kabar terakhir dari keluarga, sekitar pukul 20.30 WIB kondisinya membaik. Alat bantu oksigen sudah dilepas," ungkap Wakil Sekretaris Bidang Organisasi, Kaderisasi dan Keanggotaan DPD Partai Golkar Klaten, Purwanto saat dihubungi detikcom, Selasa (1/6/2021).

Purwanto menuturkan dari riwayat kesehatan, yang bersangkutan memang selama ini rutin kontrol ke RS di Yogyakarta. Dalam sepekan Anang dua kali kontrol dokter.

"Beliau (Anang) itu biasanya Selasa dan Jumat kontrol ke RS di Yogyakarta. Tapi sakitnya apa tidak tahu," lanjut Purwanto.

Sejak dua tahun terakhir, jelas Purwanto, Anang pernah juga dirawat di RS. Saat melihat Anang berkunjung ke gedung DPRD dirinya juga kaget.

"Pas di DPRD saya juga kaget. Saya pikir mungkin sudah sehat," imbuh Purwanto.

Purwanto menuturkan kemarin Anang ke Gedung DPRD Klaten hanya ditemani satu orang. Dia menduga kondisi Anang karena sudah mendekati jadwal kontrol.

"Ke DPRD hanya sama satu temannya. Informasinya karena mendekati jadwal kontrol kadang kondisinya menurun, jadi sudah bawa alat bantu dan tabung oksigen di mobil," ucap Purwanto yang sempat ikut mengevakuasi Anang.

Anang, ucap Purwanto, merupakan politikus senior dari Partai Golkar. Dia pun mengaku banyak mendapat telepon untuk menanyakan kabar kondisi Anang.

"Saya ditelepon banyak orang, dari provinsi dan pusat menanyakan kondisi terakhir. Beliau (Anang) termasuk salah satu guru politik saya," lanjut Purwanto.

Sebelumnya diberitakan, mantan Wakil Ketua DPRD Kabupaten Klaten, Anang Widayaka, dievakuasi dari Gedung DPRD Klaten, Senin (31/5) kemarin. Eks pimpinan DPRD dari Partai Golkar itu saat dievakuasi karena kondisinya yang tersengal-sengal saat berada di ruang Ketua DPRD Klaten Hamenang Wajar Ismoyo.

Pantauan detikcom di DPRD Klaten, politikus senior itu dievakuasi dari lantai dua, Senin (31/5) sekitar pukul 13.00 WIB. Saat dievakuasi dengan lift, Anang tampak dalam kondisi sadar tapi napasnya terlihat berat dan sudah dibantu tabung oksigen.

Anang dibawa dari ruang dinas Ketua DPRD Klaten, Hamenang Wajar Ismoyo, menggunakan kursi beroda. Dia langsung dibawa oleh mobil ambulans Pemkab Klaten menuju RSUP Dr Soeradji Tirtonegoro.'''

## RE Word$_T$

In [5]:
# https://www.geeksforgeeks.org/python-check-url-string/
# dimodifikasi agar hanya menampilkan group pertama
# TODO: Watchout, ada kasus yang menyebabkan regex lama dieksekusi
# DeprecationWarning: Flags not at the start of the expression '^((?:(?i)\\b(?:(?:htt'
URLS = r"(?i)\b(?:(?:https?:\/\/|www\d{0,3}[.]|[a-z0-9.\-]+" +\
    r"[.][a-z]{2,4}""\/)(?:[^\s()<>]+|\((?:[^\s()<>]+|(?:\(" +\
    r"?:[^\s()<>]+\)))*\))+(?:\((?:[^\s()<>]+|(?:\([^\s()<>" +\
    r"] +\)))*\)|[^\s`!()\[\]{};:'\".,<>?«»“”‘’]))"

EMAIL = r'[\w.+-]+@[\w-]+\.(?:[\w-]\.?)+[\w-]'

# Format jam umum, III.A3.
TIME = r'\d{2}\.\d{2}\.\d{2}'

# Format tanda hubung: III.E2., III.E3., III.E4., III.E5. III.E6.
HYPHEN = r'(?:[a-zA-Z]+(?:-[a-zA-Z]+)+)'
#AFFIX = KATEGLO[KATEGLO.kelas=='bt'].lema.to_list()
#AFFIX = r'\b(?:%s)' % r'|'.join(AFFIX)

# Format bilangan ribuan, III.A5.
# ditambah kemungkinan desimal, cth: 213.126,56
# kemungkinan simbol positif dan negatif tidak diurus
# ditambah bentuk umum \d+
NUMBER = r'(?:\d{4,}|(?:\d{1,3}(?:\.\d{3})*))(?:\,\d*)?'

# urus kemungkinan kasus II.H3., II.H4. II.H1.
ABBREV = r'\b(?:%s)' % r'|'.join(ABBREV)
ABBREV = ABBREV.replace(r'.', r'\.')
NAME_ABBR = r'\b[A-Z]\.'

# Format III.G2., III.I1., III.I2.
#   simbol -- sebagai subtitusi tanda pisah
#   simbol-simbol lainnya sebagai token
#   TODO: respect word boundaries
SYMBOLS = r'(?:%s)' % r'|'.join(r'''\(\?\) \.{3} -- - \n \. \, ; : \! " ' \? \( \) [ ] / —'''.split(' '))

#karakter non-spasi yang tersisa sebagai token
WORD = r'(?:[a-zA-Z]+)'
UNKNOWN = r'(?:\S+)'

In [6]:
regexes = [EMAIL, TIME, HYPHEN, NUMBER, ABBREV,
           NAME_ABBR, SYMBOLS, WORD, UNKNOWN]
PATTERN = re.compile(r'(%s)' % '|'.join(regexes), re.VERBOSE)

In [7]:
def regex_tokenize(text: str, verbose=True, del_newline=True):
    text = re.sub(r'(“|”|")', r'"', text)
    text = re.sub(r'\(\s*\?\s*\)', '(?)', text)
    if del_newline: text = text.replace('\n', ' ')

    result =  PATTERN.findall(text)
    if not verbose: return result

    tmp = []
    for token in result:
        if re.fullmatch(EMAIL, token):
            tmp.append((token, 'OBJ', 'URL'))
            continue
        if re.fullmatch(TIME, token):
            tmp.append((token, 'OBJ', 'TIME'))
            continue
        if re.fullmatch(HYPHEN, token):
            tmp.append((token, 'WORD', ''))
            continue
        #if re.fullmatch(AFFIX, token):
        #    tmp.append((token, 'WORD', ''))
        #    continue
        if re.fullmatch(NUMBER, token):
            tmp.append((token, 'OBJ', 'NUM'))
            continue
        if re.fullmatch(EMAIL, token):
            tmp.append((token, 'OBJ', 'URL'))
            continue
        if re.fullmatch(ABBREV, token):
            tmp.append((token, 'WORD', ''))
            continue
        if re.fullmatch(NAME_ABBR, token):
            tmp.append((token, 'WORD', ''))
            continue
        if re.fullmatch(SYMBOLS, token):
            tmp.append((token, 'SYMB', ''))
            continue
        if re.fullmatch(WORD, token):
            tmp.append((token, 'WORD', ''))
            continue            
        tmp.append((token, 'OBJ', ''))
    
    return tmp

In [8]:
tokens = regex_tokenize(TEXT)
len(tokens)

422

In [9]:
regex_tokenize('pasca-panen')

[('pasca-panen', 'WORD', '')]

In [10]:
AFFIX

NameError: name 'AFFIX' is not defined

## NER

## Correcting

### Stemming

In [ ]:
import sastrawi

In [ ]:
S = sastrawi.stemming.Stemmer()

In [ ]:
S.stem('menggunakan')

### Ngram spell-checking

Maybe just use `import enchant`?

In [ ]:
WORDLIST = KATEGLO.lema[KATEGLO.lema.apply(lambda x: not(' ' in x) )].to_list()

In [ ]:
def ngram(text, n=3):
    if len(text)<n: text+=' '*(n-len(text))
    return set(text[i:i+n] for i in range(len(text)-n+1))

ngram('menggunakan')

In [ ]:
%time

NGRAM = dict()

for word in WORDLIST:
    for ng in ngram(word):
        if ng not in NGRAM:
            NGRAM[ng] = set()
        NGRAM[ng].add(word)

In [ ]:
def plausible(word):
    score = {}
    
    for ng in ngram(word):
        if ng not in NGRAM:
            continue
        for word in NGRAM[ng]:
            if word not in score:
                score[word] = 0
            score[word] += 1
    iterable = iter((val, word) for word, val in score.items())
    return sorted(iterable, reverse=True)[:10]
    
def suggest(word):
    # choose if there is multiple max value...
    max_ =  plausible(word)
    if max_: max_ = max_[0]
    else: return word

    if plausible(max_[1])[0][0] - max_[0] <= 2:
        return max_[1]
    return word

plausible('Selasar')

In [ ]:
for token in tokens:
    if token[1]!='WORD' or token[0].isupper():
        #print(token[0], end=' ')
        continue

    ist = token[0].istitle()
    mask = token[0].lower()

    if mask in WORDLIST:
        #print(token[0], end=' ')
        continue
    
    sugg = suggest(token[0])
    if ist: sugg = sugg.title()

    if sugg==token[0]:
        #print(token[0], end=' ')
        continue

    print('{} : {}'.format(token[0], sugg))

### Token expansion

In [ ]:
def regex_resolve(tokens: list):
    tmp = tokens.copy()
    result = []
    
    while tmp:
        token, type_ = tmp.pop(0)
                        
        new_token = token
        # underestimate, trt to expand word
        while tmp and tmp[0][1]=='WORD':
            new_token += ' ' + tmp[0][0]
            if new_token not in DICT: break
            token = new_token
            tmp.pop(0)

        result.append((token, type_))
        
    return result

In [ ]:
regex_resolve(tokens)

## Test

In [14]:
'makan-makan' in DICT

True